# A method to correct precipitation fields for ocean models: sensitivity to degraded GPCP

*R. Dussin*

## 1. Regridding GPCP v2.3 to ERAinterim grid

The first step is to regrid the satelite-based precipitations onto the Atmospheric reanalyse grid. This can be done easily with xesmf.

In [ ]:
import xarray as xr
import numpy as np

In [ ]:
gpcpdir = './'
gpcp = xr.open_dataset(gpcpdir + 'precip.mon.mean.nc')

In [ ]:
gpcp['precip']

In [ ]:
import xesmf

In [ ]:
erai_dir = '/archive/Raphael.Dussin/ERAinterim/nc_daily'

In [ ]:
erai_grid = xr.open_dataset(f'{erai_dir}/precip_ERAinterim_1979_daily.nc', decode_times=False, drop_variables=['time', 'precip'])

In [ ]:
erai_grid

Now let's add the cell edges:

In [ ]:
# ERA-interim

lon = erai_grid['lon'].values
lon_bnds = np.concatenate((np.array([lon[0] -0.5 * 0.7031]), 0.5 * (lon[:-1] + lon[1:]), np.array([lon[-1] + 0.5 * 0.7031])), axis=0)

lat = erai_grid['lat'].values
lat_bnds = np.concatenate((np.array([-90]), 0.5 * (lat[:-1] + lat[1:]), np.array([90])), axis=0)

erai_grid['lon_b'] = xr.DataArray(data=lon_bnds, dims=('lonp1'))
erai_grid['lat_b'] = xr.DataArray(data=lat_bnds, dims=('latp1'))

In [ ]:
# GPCP

gpcp['lon_b'] = xr.DataArray(data=np.arange(0,360+2.5,2.5), dims=('lonp1'))
gpcp['lat_b'] = xr.DataArray(data=np.arange(-90,90+2.5,2.5), dims=('latp1'))

In [ ]:
gpcp2erai = xesmf.Regridder(gpcp, erai_grid, 'conservative', periodic=True)

In [ ]:
gpcp_precip_interp = gpcp2erai(gpcp['precip'])

In [ ]:
gpcp_precip_interp.sel(time='2016-1').plot()

In [ ]:
write_output= False
if write_output:
    gpcp_regridded = xr.Dataset()
    gpcp_regridded['precip'] = gpcp_precip_interp
    gpcp_regridded.to_netcdf('./GPCP_v2.3_256x512.nc')

## 1a. Degrade GPCP to annual and 5x5 degree (sensitivity tests)

In [ ]:
import calendar

In [ ]:
def weight_by_ndays(da):
    month_weights = []
    for year in list(set(da['time'].dt.year.values)):
        ndays_year = 366 if calendar.isleap(year) else 365
        for month in range(1, 13):
            _, ndays = calendar.monthrange(year, month)
            month_weights.append(float(ndays)/ndays_year)
    xr_wgts = xr.DataArray(month_weights, dims=('time'), coords={'time': da['time']})
    return xr_wgts

In [ ]:
wgts = weight_by_ndays(gpcp['precip'])

In [ ]:
# sanity check (all weights should sum to one for each year)
wgts.groupby(wgts.time.dt.year).sum(dim='time')

In [ ]:
GPCP_annual = (gpcp['precip'] * wgts).groupby(gpcp['time'].dt.year).sum(dim='time').rename({'year': 'time'})

In [ ]:
GPCP_annual.sel(time=2006).plot()

In [ ]:
GPCP_annual_interp = gpcp2erai(GPCP_annual)

In [ ]:
GPCP_annual_interp

In [ ]:
def area_regular_grid(lon, lat, Rearth=6378e3, hres=2.5):
    """ compute the cells area on a regular grid """

    rfac = 2 * np.pi * Rearth / 360
    dx1d = rfac * hres
    dy1d = rfac * hres

    dx2d, dy2d = np.meshgrid(dx1d, dy1d)
    _, lat2d = np.meshgrid(lon, lat)

    dx = dx2d * np.cos(2 * np.pi * lat2d / 360)
    dy = dy2d
    area = dx * dy
    return area

In [ ]:
area_gpcp = area_regular_grid(gpcp["lon"], gpcp["lat"], Rearth=6378e3, hres=2.5)

In [ ]:
# sanity check: global sum adds up to expected value with 0.01% relative tolerance
assert np.allclose(area_gpcp.sum(), 4*np.pi*6378e3*6378e3, rtol=0.0001)

In [ ]:
gpcp["area"] = xr.DataArray(area_gpcp, dims=('lat', 'lon'))

In [ ]:
gpcp_precip_coarsened = (gpcp["precip"] * gpcp["area"]).coarsen(lon=2, lat=2).sum() / gpcp["area"].coarsen(lon=2, lat=2).sum()

In [ ]:
gpcp_5x = gpcp_precip_coarsened.to_dataset(name='precip')
gpcp_5x['lon_b'] = xr.DataArray(data=np.arange(0,360+5,5), dims=('lonp1'))
gpcp_5x['lat_b'] = xr.DataArray(data=np.arange(-90,90+5,5), dims=('latp1'))

In [ ]:
gpcp_5x

In [ ]:
# visual check on degraded and original resolution
gpcp_5x["precip"].sel(time='2016-1').plot()

In [ ]:
gpcp["precip"].sel(time='2016-1').plot()

We need to remap the degraded GPCP to the ERAinterim grid as well:

In [ ]:
gpcp_5x_2erai = xesmf.Regridder(gpcp_5x, erai_grid, 'conservative', periodic=True)

In [ ]:
gpcp5x_precip_interp = gpcp_5x_2erai(gpcp_5x['precip'])

In [ ]:
gpcp5x_precip_interp.sel(time='2016-1').plot()

In [ ]:
write_output= True
if write_output:
    gpcp5x_regridded = xr.Dataset()
    gpcp5x_regridded['precip'] = gpcp5x_precip_interp
    gpcp5x_regridded.to_netcdf('./GPCP_v2.3_5x5deg_256x512.nc')

That's it for the input files

## 2. Generate new dataset

### Functions

In [ ]:
def cumul_precip(da):
    ''' apply cumsum and scale data array'''
    # create cumulated precip
    da_cs = da.cumsum(dim='time')
    # concat with zero initial value, needed for decumul
    zeroslice = xr.zeros_like(da_cs.isel(time=0))
    da_cs = xr.concat([zeroslice, da_cs], dim='time')
    return da_cs

def normalize_cumulated_precip(da):
    # normalize to the last value
    norm = da.isel(time=-1).clip(min=1e-15)
    da_scaled = da / norm
    return da_scaled

def decumul_precip(da):
    out = da.diff('time')
    return out

### Method

* extract monthly data from ERAinterim yearly file
* cumul/scale the data and reserve
* conservative regridding of monthly GPCP onto ERAinterim grid
* total precip in GPCP = avg monthly value * ndays_in_month
* rescale the cumulative sum with GPCP value (smoothing required?)
* run decumulation

### Compute

In [ ]:
def process_one_year(ds_erai, da_gpcp):
    current_year = ds_erai.time.dt.year[0].values
    print(current_year)
    ds_out = xr.zeros_like(ds_erai)
    for month in range(12):
        cmonth = str(month+1).zfill(2)
        data_month = ds_erai['precip'].sel(time=f'{current_year}-{cmonth}')
        ndays = len(data_month.time)
        #print(data_month.time)
        cumul = cumul_precip(data_month.clip(min=0))
        cumul_normed = normalize_cumulated_precip(cumul)
        new_total = da_gpcp.sel(time=f'{current_year}-{cmonth}').values.squeeze()
        new_total = new_total * ndays / 1000  # total precip in meters
        ny, nx = new_total.shape
        new_data_month = decumul_precip(cumul_normed.transpose(*('time', 'lat', 'lon')) * new_total) * 1000 / 86400 # kg.m-2.s-1
        if month == 0:
            da_out = new_data_month.copy()
        else:
            da_out = xr.concat([da_out, new_data_month], dim='time')
    ds_out['precip'] = da_out
    return ds_out

## Sensitivy computations (annual GPCP and GPCP 5x5 degrees)

### 5x5 degrees

In [ ]:
outdir = '/archive/Raphael.Dussin/ERAinterim/blend_GPCP_v2_sens_5x'
encoding = {'time': {'_FillValue': 0}, 'lon': {'_FillValue': 1e+36},
            'lat': {'_FillValue': 1e+36}, 'rain': {'_FillValue': 1e+36}}

encoding2 = {'time': {'_FillValue': 0}, 'lon': {'_FillValue': 1e+36},
            'lat': {'_FillValue': 1e+36}, 'precip': {'_FillValue': 1e+36}}

for year in np.arange(1979,2018+1):
    precip = xr.open_dataset(f'{erai_dir}/precip_ERAinterim_{year}_daily.nc')
    snow = xr.open_dataset(f'{erai_dir}/snow_ERAinterim_{year}_daily.nc')
    precip_corrected = process_one_year(precip, gpcp5x_precip_interp)
    # remove snow from ERAinterim to get corrected rainfall
    rain_corrected = (precip_corrected['precip'] - snow["snow"]).clip(min=0).to_dataset(name='rain')
    ds_time = xr.open_dataset(f'{erai_dir}/precip_ERAinterim_{year}_daily.nc', decode_times=False)
    precip_corrected['time'] = ds_time['time']
    precip_corrected["precip"].attrs = {'valid_min': 0., 'valid_max': 1e-2}
    precip_corrected.to_netcdf(f'{outdir}/precip_Dussin_corrected_{year}_daily_GPCP5X.nc',
                               format='NETCDF3_64BIT', encoding=encoding2)
    
    rain_corrected['time'] = ds_time['time']
    rain_corrected['rain'].attrs = {'valid_min': 0., 'valid_max': 1e-2}
    rain_corrected.to_netcdf(f'{outdir}/rain_Dussin_corrected_{year}_daily_GPCP5X.nc',
                             format='NETCDF3_64BIT', encoding=encoding)

### Annual GPCP

In [ ]:
def process_one_year_from_annual(ds_erai, da_gpcp):
    current_year = ds_erai.time.dt.year[0].values
    print(current_year)
    ds_out = xr.zeros_like(ds_erai)
    # cumul all year
    cumul = cumul_precip(ds_erai["precip"].clip(min=0))
    cumul_normed = normalize_cumulated_precip(cumul)
    new_total = da_gpcp.sel(time=current_year).values.squeeze()
    ndays = 366 if calendar.isleap(current_year) else 365
    new_total = new_total * ndays / 1000
    ny, nx = new_total.shape
    new_data = decumul_precip(cumul_normed.transpose(*('time', 'lat', 'lon')) * new_total) * 1000 / 86400 # kg.m-2.s-1
    ds_out['precip'] = new_data
    return ds_out

In [ ]:
outdir = '/archive/Raphael.Dussin/ERAinterim/blend_GPCP_v2_sens_1y'
encoding = {'time': {'_FillValue': 0}, 'lon': {'_FillValue': 1e+36},
            'lat': {'_FillValue': 1e+36}, 'rain': {'_FillValue': 1e+36}}

encoding2 = {'time': {'_FillValue': 0}, 'lon': {'_FillValue': 1e+36},
            'lat': {'_FillValue': 1e+36}, 'precip': {'_FillValue': 1e+36}}

for year in np.arange(1979,2018+1):
    precip = xr.open_dataset(f'{erai_dir}/precip_ERAinterim_{year}_daily.nc')
    snow = xr.open_dataset(f'{erai_dir}/snow_ERAinterim_{year}_daily.nc')
    precip_corrected = process_one_year_from_annual(precip, GPCP_annual_interp)
    # remove snow from ERAinterim to get corrected rainfall
    rain_corrected = (precip_corrected['precip'] - snow["snow"]).clip(min=0).to_dataset(name='rain')
    ds_time = xr.open_dataset(f'{erai_dir}/precip_ERAinterim_{year}_daily.nc', decode_times=False)
    precip_corrected['time'] = ds_time['time']
    precip_corrected["precip"].attrs = {'valid_min': 0., 'valid_max': 1e-2}
    precip_corrected.to_netcdf(f'{outdir}/precip_Dussin_corrected_{year}_daily_GPCP1Y.nc',
                               format='NETCDF3_64BIT', encoding=encoding2)
    rain_corrected['time'] = ds_time['time']
    rain_corrected['rain'].attrs = {'valid_min': 0., 'valid_max': 1e-2}
    rain_corrected.to_netcdf(f'{outdir}/rain_Dussin_corrected_{year}_daily_GPCP1Y.nc',
                             format='NETCDF3_64BIT', encoding=encoding)